In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from nltk import word_tokenize
import boto3
from s3 import get_file,upload_file
import re
import mglearn
import sklearn
import yellowbrick
from nltk.corpus import stopwords

In [ ]:
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS

In [ ]:
s3 = boto3.resource('s3')
lyrics = get_file(s3,'s3ssp',download_file='NLP_Data/rough_lyrics_audio_features.csv',rename_file='0320_2020_nlp.csv')

In [ ]:
df = pd.read_csv(lyrics,sep='|',encoding='utf-8')
df.head()
df = df.drop_duplicates(['artist_uri_x','track_uri'])
df_copy = df.copy().dropna()

## Clean of stop words from NLTK and Github Song Stopwords Resource

In [ ]:
df_copy.head(1)

In [ ]:
df_lastfm_stopwords = pd.read_csv('stopwords.txt',header=None)

In [ ]:
lyrics_list = df_copy['lyrics'].str.replace('\n',' ')
df_copy['lyrics'] = lyrics_list.to_frame()
lastfm_stopwords = df_lastfm_stopwords[0].str.replace('\n',' ')
df_lastfm_stopwords['last_fm_stopwords'] = lastfm_stopwords.to_frame()
#df_clean = .str.replace('(\[.*\])*','')

In [ ]:
df_lastfm_stopwords.head()

In [ ]:
df_lastfm_stopwords_list = df_lastfm_stopwords['last_fm_stopwords'].tolist()

In [ ]:
type(df_lastfm_stopwords_list)

In [ ]:
df_english = pd.read_csv("english.txt",header=None)

In [ ]:
github_english_stopwords = df_english[0].str.replace('\n',' ')
df_english['github_english_stopwords'] = github_english_stopwords.to_frame()
github_english_stopwords_list = df_english['github_english_stopwords'].tolist()

In [ ]:
type(github_english_stopwords_list)

In [ ]:
df_spanish = pd.read_csv("spanish.txt",header=None)
github_spanish_stopwords = df_spanish[0].str.replace('\n',' ')
df_spanish['github_spanish_stopwords'] = github_spanish_stopwords.to_frame()
github_spanish_stopwords_list = df_spanish['github_spanish_stopwords'].tolist()

In [ ]:
type(github_spanish_stopwords_list)

## Clean from Instrumentals

In [ ]:
#Clean dataset of instrumentals
df_playlist = df_copy.copy()

df_playlist['instrumental'] = df_playlist['lyrics'].apply(lambda x: x.find('instrumental'))
df_playlist = df_playlist.drop(df_playlist[df_playlist['instrumental'] != -1].index)
#df = df.drop(df[df['lyrics']=='This is an instrumental'].index)

In [ ]:
df_playlist['Instrumental'] = df_playlist['lyrics'].apply(lambda x: x.find('Instrumental'))
df_playlist =df_playlist.drop(df_playlist[df_playlist['Instrumental'] != -1].index)

In [ ]:
df_playlist['INSTRUMENTAL'] = df_playlist['lyrics'].apply(lambda x: x.find('INSTRUMENTAL'))
df_playlist =df_playlist.drop(df_playlist[df_playlist['INSTRUMENTAL'] != -1].index)

In [ ]:
df_playlist[df_playlist['lyrics'] == 'INSTRUMENTAL']

## Clean stopwords and punctuation

In [ ]:
def preprocessor(text,source):
    tokens = word_tokenize(text)
    return (" ").join([word for word in tokens if word not in source])

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:
df_demo = df_playlist

df_demo['git_english_lyrics'] = df_demo['lyrics'].apply(lambda x: preprocessor(x,github_english_stopwords_list))
df_demo['git_spanish_lyrics'] = df_demo['git_english_lyrics'].apply(lambda x: preprocessor(x,github_spanish_stopwords_list))
df_demo['last_fm_lyrics'] = df_demo['git_spanish_lyrics'].apply(lambda x: preprocessor(x,df_lastfm_stopwords_list))
df_demo['nltk_lyrics'] = df_demo['last_fm_lyrics'].apply(lambda x: preprocessor(x,stopwords.words('english')))




In [ ]:
df_demo.head()


## Clean Lyrics Dataset

In [ ]:
df_topic = df_demo[['artist_name','track_name','track_uri','nltk_lyrics','acousticness','danceability','energy','valence'
                    ,'instrumentalness','key','mode','liveness','speechiness','time_signature',
                    'tempo','loudness']]
df_topic = df_topic.rename(columns={'nltk_lyrics':'lyrics'})
df_topic.describe()

In [ ]:
#df_topic[(df_topic['artist_name']=='Arcade Fire') & (df_topic['track_name']=='here comes the night time')]

In [ ]:
df_topic.to_csv('master_lyrics_audio_features.csv',index=False,sep='|')

## Upload file to S3

In [ ]:
upload_file(s3,'s3ssp',upload_file='master_lyrics_audio_features.csv',rename_file='NLP_Data/master_lyrics_audio_features.csv')